# CLEAN DATA

Remove the galactic dust and the galactic dipole from RAW data; this is necessary to make a correct classification of the glitches.

In [1]:
import healpy as hp
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import statistics as sts
from scipy.linalg import lstsq

## Model constants and parameters

In [2]:
SPEED_OF_LIGHT_M_S = 2.99792458e8
PLANCK_H_MKS = 6.62606896e-34
BOLTZMANN_K_MKS = 1.3806504e-23
SOLSYSSPEED_M_S = 370082.2332
SOLSYSDIR_ECL_COLAT_RAD = 1.7656051330336222
SOLSYSDIR_ECL_LONG_RAD = 2.9958842149922833
T_CMB = 2.72548

SOLSYS_SPEED_VEC_M_S = SOLSYSSPEED_M_S * np.array(
                                                  [
                                                   np.sin(SOLSYSDIR_ECL_COLAT_RAD) * np.cos(SOLSYSDIR_ECL_LONG_RAD),
                                                   np.sin(SOLSYSDIR_ECL_COLAT_RAD) * np.sin(SOLSYSDIR_ECL_LONG_RAD),
                                                   np.cos(SOLSYSDIR_ECL_COLAT_RAD),
                                                   ]
                                                  )

In [3]:
OPERATING_DAY = "091"
FILENAME_PTG = "/mnt/d/Tesi/data/HFI-143/HFI_TOI_143-PTG_R2.01_OD0" + OPERATING_DAY + ".fits"
FILENAME_RAW = "/mnt/d/Tesi/data/HFI-143/HFI_TOI_143-RAW_R2.00_OD0" + OPERATING_DAY + ".fits"
FILENAME_SCI = "/mnt/d/Tesi/data/HFI-143/HFI_TOI_143-SCI_R2.00_OD0" + OPERATING_DAY + ".fits"

DETECTOR = "143-5"

FILENAME_MASK = "ris/HFI_DustMask_0.009.fits.gz"

## Cleaning model

Since the purpose of this thesis is to detect glitches and not to clean up the RAW signal from the galactic signal, all the points found on the galactic plane can be ignored without any consequences.

### SCI data flags

Load galaxy flag from the SCI data. The SCI data, taken as PTG and RAW data from the Planck Legacy Archive (PLA), are the so-called scientific data (already cleaned of various effects and glitches) and each data has a flag that indicates a peculiarity, i.e. point object, planet, galaxy plane and others. In particular, the flag of interest is that concerning the galaxy:
```
bit 4: StrongSignal; 1 = In Galactic plane
```
Data with this flag are localized on the galactic plane and must therefore be discarded.

In [4]:
# Open SCI data and load the "FLAG" field
with fits.open(FILENAME_SCI) as f:
    SCI_FLAG_bits = f[DETECTOR].data.field("FLAG")
    
# Stack array in sequence vertically
#SCI_FLAG_bits = np.vstack(SCI_FLAG_bits)
# Unpacks bits
SCI_FLAG_bits = np.unpackbits(SCI_FLAG_bits[:, np.newaxis], axis=1)
# Read the 4th bit (aka the 4th column of the multidimensional array)
SCI_FLAG = SCI_FLAG_bits[:,3]

### Dust mask

Load the dust mask previously created.

In [5]:
# Read the dust mask
MASK = hp.read_map(FILENAME_MASK, verbose=False)
# The nside (number of pixels) is taken as that of the mask
NSIDE = hp.npix2nside(len(MASK))

# Open PTG data and load the "THETA" and "PHI" fields
with fits.open(FILENAME_PTG) as inpf:
    theta, phi = [inpf[DETECTOR].data.field(x) for x in ("THETA", "PHI")]

In [6]:
# Convert the sequence of positions in the sky into a sequence of pixel indexes
# Not used because creates a problem with the averaging
#pixidx = hp.ang2pix(NSIDE, theta, phi)

# Get the directions (vectors) directly from the angular coordinates
directions = hp.ang2vec(theta, phi)
# Take of the average of two consecutive directions
directions = (directions[:-1:2] + directions[1::2]) / 2
# Convert the directions into pixels - taking directly from the angles creates a problem with the averaging
pixidix = hp.vec2pix(NSIDE ,directions[:,0], directions[:,1], directions[:,2])

# Create array with 1s and 0s corresponding to the positions "good" or "bad"
MASK_FLAG = MASK[pixidix] == 0
# Make an array with INT type out of the bool one.
MASK_FLAG = MASK_FLAG.astype(np.int)

### RAW data preparation

In [7]:
# Open the voltages
with fits.open(FILENAME_RAW) as f:
    obt = f["OBT"].data.field("OBT")
    data_raw = f[DETECTOR].data.field("RAW")

#### Time

First the OBT (_on board time_) must be converted to seconds and the offset must be removed, then take the average between two consecutive times. The conversion constant is `65536`

In [8]:
# Convert the time from OBT clock to seconds and remove the offset
time = (obt - obt[0]) / 65536
# Average the time of two adjacent samples
time = (time[:-1:2] + time[1::2]) / 2

#### Data

Take the average between two consecutive data. 

In [9]:
#data2 = (data[1::2] - data[:-1:2])/2
data = np.abs(data_raw)
data = (data[:-1:2] + data[1::2])/2

#### Galactic dipole

Compute the galactic dipole.

`get_dipole_temperature(directions)` : given one or more one-length versors, return the intensity of the CMB dipole; vectors must be expressed in the ecliptic coordinate system.

In [10]:
def get_dipole_temperature(directions):
    
    beta = SOLSYS_SPEED_VEC_M_S / SPEED_OF_LIGHT_M_S
    gamma = (1 - np.dot(beta, beta)) ** (-0.5)
    
    return T_CMB * (1.0 / (gamma * (1 - np.dot(directions, beta))) - 1.0)

In [11]:
dipole = get_dipole_temperature(directions)

### Linear regression

In [ ]:
# Calculate the medians for the data and the dipole temperatures and rescale accordingly
median_V = sts.median(data)
median_D = sts.median(dipole)

data = data - median_V
dipole = dipole - median_D

Adesso viene effettuata una regressione lineare... WHY??

In [ ]:


# Evaluate the G factor, such as V - median(V) = G • (T - median(T))
M = dipole[:, np.newaxis]*[0, 1]
p, res, rnk, s = lstsq(M, data)
G = p[1]

# Take the dipole out of the data
V_CORRECT = data - G*dipole

# Take out from the data the directions corresponding to the galactic dust mask
clean_data = V_CORRECT[MASK_FLAG==1]
holed_raw = data[MASK_FLAG==1]

# Take out also on the time - this way I can have "holes" in the graph
time_clean = time[MASK_FLAG==1]

np.savetxt("data_to_classify/time_clean.txt", time_clean)
np.savetxt("data_to_classify/clean_data.txt", clean_data)

In [ ]:



#
#
#
# PLOTTING -- IF DO NOT WANT PLOT, REMOVE THIS PART
#
#
#

mask = time < 30.

# plot the dipole
plt.figure(1)
plt.plot(time[mask], dipole[mask], marker='.', linestyle='none', color='#d9b83b')
plt.xlabel("Time [s]")
plt.ylabel("Temperature [K]")
plt.title("Dipole temperatures")
plt.savefig("plots/dipole_example.png", dpi=600)

#plot the data - both raw and cleaned (two times, once without holes and ones with holes)
plt.figure(2)
plt.plot(time[mask], data[mask], marker='.', linestyle='none', color='#4496e7', alpha=0.9, label="with dipole")
plt.plot(time[mask], V_CORRECT[mask], marker='.', linestyle='none', color='#df413a', alpha=0.9, label="without dipole")
plt.title("Before and after dipole removal signal — with galactic dust.")
plt.xlabel("Time [s]")
plt.ylabel("Signal")
plt.legend("best")

plt.savefig("plots/gal_signal.png", dpi=600)

plt.figure(3)
mask = time_clean < 30.
plt.plot(time_clean[mask], holed_raw[mask], marker='.', linestyle='none', color='#4496e7', alpha=0.9, label="with dipole")
plt.plot(time_clean[mask], clean_data[mask], marker='.', linestyle='none', color='#df413a', alpha=0.9, label="without dipole")
plt.title("Before and after dipole removal signal — without galactic dust.")
plt.xlabel("Time [s]")
plt.ylabel("Signal")
plt.legend("best")

plt.savefig("plots/no_gal_signal.png", dpi=600)

